In [2]:
import sys
print(f'Python version:', sys.version)

Python version: 3.6.8 (tags/v3.6.8:3c6b436a57, Dec 24 2018, 00:16:47) [MSC v.1916 64 bit (AMD64)]


In [39]:
# Packages
import termcolor as tc
import Bio; print('BioPython version:', Bio.__version__)

# Python packages
import os
import sys
import gc
import string
import pickle
import collections
import random

# Data Science and Computation
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import scipy # statistics
import sklearn # machine learning

# Natural language processing
import nltk

# Visualization 
import matplotlib.pyplot as plt
import seaborn as sns

# Local Imports
import definitions as defs

BioPython version: 1.76


In [16]:
sars_file = os.path.join(defs.DATA_DIR, 'sars_sequences.fasta')

In [17]:
sars_file

'D:\\code\\py\\corona-virus-analysis\\data\\sars_sequences.fasta'

# Helper Functions

In [41]:
def examine_distribution(sequence, title=''):
    # Calculate statistics
    mean = np.mean(sequence)
    median = np.median(sequence)
    mode = int(scipy.stats.mode(sequence).mode)
    statistics = {'Mean': mean, 'Median': median, 'Mode': mode}

    # Plot distribution
    fig, ax = plt.subplots()
    sns.distplot(sequence)
    ax.axvline(mean, color='r', linestyle='--')
    ax.axvline(median, color='g', linestyle='-')
    ax.axvline(mode, color='b', linestyle='-')

    plt.title(title.capitalize())
    plt.legend(statistics)
    plt.show()

    for key, value in statistics.items():
        print(f'\t{key}={value:.2f}')


In [7]:
handle = Entrez.efetch(db='nucleotide', id='1850859292', rettype='fasta', retmode='text')
record_b = SeqIO.read(handle, 'fasta')

In [31]:
from Bio import SeqIO
from Bio import Entrez

Entrez.email = 'filip.markoski45@gmail.com'
handle = Entrez.esearch(db='nucleotide', term='SARS-CoV-2')
search = Entrez.read(handle)

for id in search['IdList']:
    handle = Entrez.efetch(db='nucleotide', id=id, rettype='fasta', retmode='text')
    record = SeqIO.read(handle, 'fasta')
    print(record.description)
    print(len(record.seq))

2

# Code

In [37]:
?? Entrez.efetch

In [320]:
def construct_features(seq):
    features = []
    
    # quick features
    features.append(len(seq))
    
    features.extend(construct_sequtils_features(seq, window=500))
    features.extend(construct_nucleotide_counts(seq, display=False))
    features.extend(construct_amino_acid_features(seq, display=False))
    features.extend(construct_n_gram_features(seq, window_size=3))
    features.extend(construct_fourier_wavelet_feature(seq))
    
    return np.array(features)

In [321]:
features = []

with open(sars_file) as fasta_file:  # Will close handle cleanly
    identifiers = []
    lengths = []
    nucleotides = []
    descriptions = []
    rna = []
    amino_acid = []
    count = 0
    for seq_record in SeqIO.parse(fasta_file, 'fasta'):  # (generator)
        if len(set(seq_record.seq)) != 4:
            continue 
            
        if count == 5:
            break
        count+=1
        
        identifiers.append(seq_record.id)
        lengths.append(len(seq_record.seq))
        nucleotides.append(seq_record.seq)
        descriptions.append(seq_record.description)
        rna_temp = seq_record.seq.transcribe()
        rna.append(rna_temp)
        amino_acid.append(rna_temp.translate())
        #print(set(seq_record.seq.translate()), 'yes')
        features.append(construct_features(seq_record.seq))

        
s1 = pd.Series(identifiers, name='ID')
s2 = pd.Series(lengths, name='length')
#s3 = pd.Series(nucleotides, name='Seq')
s4 = pd.Series(descriptions, name='Description')
#Gathering Series into a pandas DataFrame and rename index as ID column
sars_nucelotide = pd.DataFrame(dict(ID=s1, length=s2, Seq=nucleotides,Description=s4, RNA=rna, Amino_Acid=amino_acid)).set_index(['ID'])
sars_nucelotide

D:\code\py\corona-virus-analysis\.env\lib\site-packages\Bio\Seq.py:2859: BiopythonWarning: Partial codon, len(sequence) not a multiple of three. Explicitly trim the sequence or add trailing N before translation. This may become an error in future.
  BiopythonWarning,


,length,Seq,Description,RNA,Amino_Acid
ID,,,,,
KJ473811.1,29037,"(G, G, T, T, T, C, G, T, C, C, G, G, G, T, G, ...",KJ473811.1 |BtRf-BetaCoV/JL2012| complete geno...,"(G, G, U, U, U, C, G, U, C, C, G, G, G, U, G, ...","(G, F, V, R, V, *, P, K, G, K, M, E, S, L, V, ..."
FJ588686.1,29059,"(C, C, A, G, G, A, A, A, A, G, C, C, A, A, C, ...",FJ588686.1 |Bat SARS CoV Rs672/2006| complete ...,"(C, C, A, G, G, A, A, A, A, G, C, C, A, A, C, ...","(P, G, K, A, N, Q, P, R, S, L, V, D, L, F, S, ..."
KJ473816.1,29142,"(G, T, T, T, C, G, T, C, C, G, G, G, T, G, T, ...",KJ473816.1 |BtRs-BetaCoV/YN2013| complete geno...,"(G, U, U, U, C, G, U, C, C, G, G, G, U, G, U, ...","(V, S, S, G, C, D, R, K, V, R, W, R, A, L, F, ..."
KJ473815.1,29161,"(G, T, C, C, G, G, G, T, G, T, G, A, C, C, G, ...",KJ473815.1 |BtRs-BetaCoV/GX2013| complete geno...,"(G, U, C, C, G, G, G, U, G, U, G, A, C, C, G, ...","(V, R, V, *, P, K, G, K, M, E, S, L, V, L, G, ..."
KY352407.1,29274,"(T, A, A, A, A, G, G, A, T, T, A, A, T, C, C, ...",KY352407.1 |Severe acute respiratory syndrome-...,"(U, A, A, A, A, G, G, A, U, U, A, A, U, C, C, ...","(*, K, D, *, S, F, P, E, N, P, T, N, L, D, L, ..."


In [323]:
sars_features = pd.DataFrame(features).fillna(0)
sars_features['ID'] = s1
sars_features = sars_features.set_index('ID')
sars_features

,0,1,2,3,4,5,6,7,8,9,...,29387,29388,29389,29390,29391,29392,29393,29394,29395,29396
ID,,,,,,,,,,,,,,,,,,,,,
KJ473811.1,29037.0,40.999415,9.311954e+06,0.235294,-0.103825,0.024313,0.026786,-0.051163,8254.0,8878.0,...,0.00000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
FJ588686.1,29059.0,41.092261,9.319131e+06,0.400000,-0.164706,0.024767,0.025126,0.000000,8266.0,8852.0,...,0.00000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
KJ473816.1,29142.0,41.102189,9.345598e+06,0.251185,-0.134328,0.022514,0.027027,0.054187,8230.0,8934.0,...,0.00000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
KJ473815.1,29161.0,40.862796,9.352055e+06,0.241706,-0.156250,0.019012,0.024631,0.000000,8342.0,8903.0,...,0.00000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
KY352407.1,29274.0,39.215686,9.386117e+06,0.187817,-0.123596,0.055701,0.056995,0.025381,8354.0,9440.0,...,-2.12132,1.414214,0.707107,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [328]:
location = os.path.join(defs.DATA_DIR, 'sars_features.csv')
sars_features.to_csv(location)

# Features 

In [298]:
from Bio.SeqUtils import *

def construct_sequtils_features(seq, window=500, display=False) -> list:
    seq = seq.transcribe()
    
    gc_content = GC(seq)
    
    mol_weight = molecular_weight(seq, seq_type='RNA')
    
    gc_skew_array = GC_skew(seq, window=window)
    
    if display:
        print(len(gc_skew_array))

    max_gc_skew = np.max(gc_skew_array)
    min_gc_skew = np.min(gc_skew_array)
    mean = np.mean(gc_skew_array)
    median = np.median(gc_skew_array)
    mode = scipy.stats.mode(gc_skew_array).mode[0]

    vector = [gc_content, mol_weight, max_gc_skew, min_gc_skew, mean, median, mode]
    # vector.extend(gc_skew_array[:60])
    return vector

len(construct_sequtils_features(seq=record.seq))

7

In [241]:
#count feature
def construct_nucleotide_counts(seq, display=False) -> list:
    seq = seq.transcribe()
    
    mapping = {
        'A': 0,
        'U': 0,
        'G': 0,
        'C': 0,
    }
    template = collections.Counter(mapping)
    counter = collections.Counter(seq)
    template =  template + counter
    
    if display:
        print(list(template.items()))
    
    vector = list(template.values())
    return vector

construct_nucleotide_counts(record.seq, display=True)

[('A', 8954), ('U', 9603), ('G', 5860), ('C', 5486)]


[8954, 9603, 5860, 5486]

In [240]:
#amino-acid distribution

def construct_amino_acid_features(seq, display=False) -> list:
    seq_AminoAcids = seq_rna.translate()
    amino_acids_dict = {'Y': 0, 'A': 0, 'Q': 0, 'D': 0, 'C': 0, 'G': 0, 'V': 0, 'T': 0, 'E': 0, 'N': 0, 
                       'K': 0, 'R': 0, 'S': 0, 'I': 0, 'H': 0, 'M': 0, 'F': 0, 'L': 0, 'W': 0, 'P': 0}
    
    # Percentages
    
    for amino_acid in amino_acids_dict:
        amino_acids_dict[amino_acid] = seq_AminoAcids.count(amino_acid)/len(seq_AminoAcids)*100
    
    vector = list(amino_acids_dict.values())
    
    if display:
        print(amino_acids_dict.values())
    
    # Counts 
    
    for amino_acid in amino_acids_dict:
        amino_acids_dict[amino_acid] = seq_AminoAcids.count(amino_acid)
    
    vector.extend(list(amino_acids_dict.values()))
    return vector
        
len(construct_amino_acid_features(record.seq, display=False))

40

In [238]:
import itertools

def construct_n_gram_features(seq, window_size=3) -> list:
    permutations = itertools.product('ACTG', repeat=window_size)
    mapping = dict.fromkeys(list(permutations), 0)

    template = collections.Counter(mapping)

    n_gram_generator = ngrams(record.seq, window_size)
    counter = collections.Counter(n_gram_generator)

    template = template + counter
    
    # getting the feature name
    # ''.join(list(counter.keys())[0])
    
    vector = list(template.values())
    return vector
    
print(len(construct_n_gram_features(record.seq, window_size=3)))

64


In [61]:
''.join(list(counter.keys())[0])

'ATT'

In [312]:
import operator
import functools
import pywt
import math
#https://pywavelets.readthedocs.io/en/0.2.2/ref/dwt-discrete-wavelet-transform.html
#mapping functon
def toi(n):
    if n=='A':
        return 0
    if n=='C':
        return 1
    if n=='G':
        return 2
    if n=='U':
        return 3
    
# determine the decomposition level
# N denotes the length of the DNA sequence
# M is the fixed length of the feature vector, and L is the decomposition level.
def decomposition_lvv(N,M):
    r = N/M
    ll = math.log(r,2)
    L = math.ceil(ll)
    return L    

# DWT reduces the dimension of CODE. WFV uses the simplest Haar wavelet to create the feature vector of DNA sequence S
def construct_fourier_wavelet_feature(seq):
    # convert to RNA sequence
    seq = seq.transcribe()
 
    code_list = list(map(toi, seq)) 
    code = ''.join(map(str, code_list))

    if len(code) < 30133:
        number = 30133 - len(code)
        s = ''.zfill(number)
        code += s
  
        
    N = len(code)
    l = decomposition_lvv(N, 2)
    
    coeffs = pywt.wavedec(code_list, 'haar', level = l)
    
    # return list(np.array(coeffs).flatten())
    
    # flattening the list
    coeffs =functools.reduce(operator.iconcat, coeffs, [])
    
    # return list(np.array(coeffs).flatten())
    
    return list(coeffs)
    

print(len(construct_fourier_wavelet_feature(a)))

29909


In [212]:

a = record.seq
b = x
result1 = construct_fourier_wavelet_feature(a)
result2 = construct_fourier_wavelet_feature(b)
print(len(result1))
print(len(result2))
for i in range(0,len(result1)):
    if len(result1[i]) == len(result2[i]):
        print(i, 'True')

Seq('AAC')

In [230]:
s= ''
s.zfill(4)

'0000'